In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import spectrum_utils.plot as sup
import spectrum_utils.spectrum as sus
import urllib.parse
import bio_helper
import tools
import seaborn as sns
from tools import *
%load_ext autoreload
%autoreload 2
from importlib import reload
reload(bio_helper)
plt.style.use([])
import sys
sys.path.append("..")
from ms import finetune
from fdr_test import fixed_features

In [2]:
which = "Mel1"
msms_file = f"/data/yejb/prosit/figs/boosting/figs/Figure_5_{which}/forPride/txt/msms.txt"
raw_dir =   f"/data/yejb/prosit/figs/boosting/figs/Figure_5_{which}/forPride/rescoring_for_paper_2/raw"
fixed_features_dir = f"/data/yejb/prosit/figs/boosting/figs/Figure_5_{which}/fix_features/"

name, msms_data = tools.read_msms(msms_file)
msms_data = tools.filter_msms(name, msms_data)


msms_data = msms_data[:100000]
msms_data.sort(key=lambda x: int(x[name.index("id")]))
m_r = loc_msms_in_raw(msms_data, raw_dir)
tabels_file = fixed_features(
            msms_file, raw_dir, fixed_features_dir)


In [3]:
peptides = [i[0][name.index("Sequence")].replace("_", "") for i in m_r]
charges = [int(i[0][name.index("Charge")]) for i in m_r]
precursor_mz = [float(i[0][name.index("m/z")]) for i in m_r]
andromeda = [float(i[0][name.index("Score")]) for i in m_r]
labels = [-1 if len(i[0][name.index("Reverse")]) else 1 for i in m_r] 

In [4]:
from ms import model
import ms

run_model = model.PrositFrag()
run_model.load_state_dict(torch.load(
    "../checkpoints/frag_boosting/best_hcd_frag_PrositFrag-512.pth", map_location="cpu"))
prosit = run_model.eval()

finetuned_prosit, _ = finetune.semisupervised_finetune(prosit, tabels_file, gpu_index=2)

Run on cuda:2
Baseline Andromeda:
8231 59796 335683
826 120243 433010
Achieve best


KeyboardInterrupt: 

In [ ]:
matched_ions_pre = generate_matched_ions(m_r)
m_i = [bio_helper.match_all(i, 'yb') for i in tqdm(matched_ions_pre)]
frag_msms = [bio_helper.reverse_annotation(*i[:4]) for i in tqdm(m_i)]

100%|██████████| 96074/96074 [00:04<00:00, 21228.91it/s]


In [ ]:
data_nce_cand = generate_from_mi(matched_ions_pre, name, nces=33)
prosit_sa, prosit_inten = get_sa_all(prosit, data_nce_cand, frag_msms, gpu_index=2)
prosit_sa = prosit_sa.cpu().numpy()
prosit_inten = prosit_inten.cpu().numpy()

finetune_prosit_sa, finetune_prosit_inten = get_sa_all(finetuned_prosit, data_nce_cand, frag_msms, gpu_index=2)
finetune_prosit_sa = finetune_prosit_sa.cpu().numpy()
finetune_prosit_inten = finetune_prosit_inten.cpu().numpy()

In [ ]:
import matplotlib as mpl
plt.style.use(['ieee', 'no-latex'])
plt.rcParams["font.family"] = "Times New Roman"
joint_pd = pd.DataFrame({"SA": prosit_sa, "Fine-tuned SA": finetune_prosit_sa,
                        "label": ['decoy' if m==-1 else "target" for m in labels]})
ax = sns.jointplot(x="SA", y='Fine-tuned SA', data=joint_pd, hue='label')
mpl.rcParams.update(mpl.rcParamsDefault)


In [ ]:
combined_scores = [(i,p) for i, p in enumerate(zip(andromeda, prosit_sa, finetune_prosit_sa, labels))]
combined_scores.sort(key=lambda x: x[1][2] - x[1][1])

for p in combined_scores[-200:-100]:
    print(f"{p[0]}({p[1][3]}): {p[1][0]},\t{p[1][2] - p[1][1]:.4f}")

52744(1): 144.38,	0.4181
95875(-1): 20.282,	0.4188
20886(1): 67.464,	0.4188
32272(1): 105.14,	0.4196
8949(1): 95.741,	0.4199
72600(1): 14.776,	0.4202
36645(-1): 6.0316,	0.4202
50952(1): 38.202,	0.4207
63571(1): 38.202,	0.4211
92099(-1): 6.1628,	0.4211
26885(1): 138.24,	0.4215
14913(1): 62.546,	0.4234
45950(1): 76.035,	0.4240
652(-1): 10.232,	0.4257
23163(-1): 7.2299,	0.4264
57622(1): 158.36,	0.4266
86841(1): 38.202,	0.4271
13245(1): 23.071,	0.4272
21428(1): 31.708,	0.4273
78614(1): 29.376,	0.4275
44321(1): 89.355,	0.4282
2796(1): 120.65,	0.4285
80808(1): 116.73,	0.4289
69506(1): 38.202,	0.4289
26515(1): 38.202,	0.4293
64474(1): 21.547,	0.4294
20229(1): 69.825,	0.4297
31656(1): 19.562,	0.4298
68668(1): 10.138,	0.4303
7979(1): 7.7117,	0.4311
33306(1): 44.349,	0.4314
57140(1): 138.54,	0.4315
63432(1): 138.54,	0.4316
62767(1): 9.3474,	0.4324
62438(-1): 11.411,	0.4327
1676(1): 9.3474,	0.4329
36114(1): 6.931,	0.4337
86673(1): 124.98,	0.4341
57663(1): 129.68,	0.4348
80953(1): 38.202,	0.4351
2

In [ ]:
import spectrum_vis as sup
import matplotlib.ticker as mticker

plt.rcParams["font.family"] = "Ubuntu"

which_psm = 52744
intensity = m_i[which_psm][1]
mass = m_i[which_psm][5]
peptide = peptides[which_psm]
m_over_z = precursor_mz[which_psm]
charge = charges[which_psm]
pred_inten = prosit_inten[which_psm]
pred_inten[pred_inten<0] = 0
sa = prosit_sa[which_psm]
fine_pred_inten = finetune_prosit_inten[which_psm]
fine_pred_inten[fine_pred_inten<0] = 0
fine_sa = finetune_prosit_sa[which_psm]

pred_match = bio_helper.pair_backbone_with_mass(pred_inten, peptide, charge)
fine_pred_match = bio_helper.pair_backbone_with_mass(fine_pred_inten, peptide, charge)


# print(pred_match[0], pred_match[1])
spectrum_1 = sus.MsmsSpectrum("see", m_over_z, charge, mass, intensity, peptide=peptide) .filter_intensity(min_intensity=0.05, max_num_peaks=50)
spectrum_2 = sus.MsmsSpectrum("see", m_over_z, charge, pred_match[1], pred_match[0], peptide=peptide) .filter_intensity(min_intensity=0.05, max_num_peaks=50)
spectrum_3 = sus.MsmsSpectrum("see", m_over_z, charge, fine_pred_match[1], fine_pred_match[0], peptide=peptide) .filter_intensity(min_intensity=0.05, max_num_peaks=50)

fig, ax = plt.subplots(figsize=(9, 6), dpi=300)
fragment_tol_mass = 30
fragment_tol_mode = 'ppm'
spectrum_1 = spectrum_1.annotate_peptide_fragments(fragment_tol_mass, fragment_tol_mode, ion_types='by')
spectrum_2 = spectrum_2.annotate_peptide_fragments(fragment_tol_mass, fragment_tol_mode, ion_types='by')
spectrum_3 = spectrum_3.annotate_peptide_fragments(fragment_tol_mass, fragment_tol_mode, ion_types='by')
# sup.mirror(spectrum_1, spectrum_2)
gd_color = {None: "#212121"}
before_color = {None: "#212121"}
after_color = {None: "#F57C00"}

sup.spectrum(spectrum_1, grid=False, annotate_ions=True, color_ions=False,linewidth=8, alpha=0.2, ax=ax, colors=gd_color, label="Observed")
sup.spectrum(spectrum_2, grid=False, ax=ax, annotate_ions=True, color_ions=True, colors=before_color, label="No fine-tuned")

sup.spectrum(spectrum_1, grid=False, annotate_ions=False, color_ions=False,linewidth=8, alpha=0.2, mirror_intensity=True, ax=ax)
sup.spectrum(spectrum_3, grid=False, mirror_intensity=True, color_ions=True, ax=ax, annotate_ions=True, colors=after_color, label="Fine-tuned")
ax.set_ylim(-1.5, 1.5)
ax.set_xlim(-100, 1000)
ax.axhline(0, color="#9E9E9E", zorder=10)
ax.yaxis.set_major_locator(mticker.AutoLocator())
ax.yaxis.set_minor_locator(mticker.AutoMinorLocator())
ax.yaxis.set_major_formatter(
    mticker.FuncFormatter(lambda x, pos: f"{abs(x):.0%}")
)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=5)
ax.text(-90, 1.2, f"Andromeda: {andromeda[which_psm]}\nSA: {sa:.3f}", fontsize=14)
ax.text(-90, -1.4, f"Andromeda: {andromeda[which_psm]}\nSA: {fine_sa:.3f}", fontsize=14)
print(andromeda[which_psm],sa, fine_sa)
